In [1]:
import os
from pathlib import Path
import sys

In [2]:
os.getcwd()

'/mnt/4TB1/onj/onj_project_template/preprocess'

In [3]:
DATAPATH = Path("../../dataset/v0/")

In [4]:
patient_path = DATAPATH / "ONJ" / "EW-0003" / "MDCT" / "20191212" / "MDCT_axial"
print(patient_path)

../../dataset/v0/ONJ/EW-0003/MDCT/20191212/MDCT_axial


In [5]:
import dicom2nifti

In [6]:
os.listdir(str(patient_path / "dicom"))

['MD157.dcm',
 'MD62.dcm',
 'MD127.dcm',
 'MD14.dcm',
 'MD134.dcm',
 'MD50.dcm',
 'MD27.dcm',
 'MD8.dcm',
 'MD30.dcm',
 'MD102.dcm',
 'MD67.dcm',
 'MD69.dcm',
 'MD173.dcm',
 'MD174.dcm',
 'MD12.dcm',
 'MD131.dcm',
 'MD97.dcm',
 'MD73.dcm',
 'MD48.dcm',
 'MD167.dcm',
 'MD2.dcm',
 'MD40.dcm',
 'MD114.dcm',
 'MD71.dcm',
 'MD21.dcm',
 'MD31.dcm',
 'MD76.dcm',
 'MD150.dcm',
 'MD172.dcm',
 'MD13.dcm',
 'MD132.dcm',
 'MD140.dcm',
 'MD64.dcm',
 'MD74.dcm',
 'MD3.dcm',
 'MD6.dcm',
 'MD149.dcm',
 'MD146.dcm',
 'MD0.dcm',
 'MD155.dcm',
 'MD141.dcm',
 'MD111.dcm',
 'MD23.dcm',
 'MD183.dcm',
 'MD117.dcm',
 'MD169.dcm',
 'MD145.dcm',
 'MD46.dcm',
 'MD133.dcm',
 'MD34.dcm',
 'MD104.dcm',
 'MD153.dcm',
 'MD45.dcm',
 'MD103.dcm',
 'MD98.dcm',
 'MD60.dcm',
 'MD170.dcm',
 'MD81.dcm',
 'MD118.dcm',
 'MD66.dcm',
 'MD135.dcm',
 'MD123.dcm',
 'MD47.dcm',
 'MD136.dcm',
 'MD44.dcm',
 'MD95.dcm',
 'MD120.dcm',
 'MD166.dcm',
 'MD121.dcm',
 'MD75.dcm',
 'MD105.dcm',
 'MD122.dcm',
 'MD28.dcm',
 'MD68.dcm',
 'MD5.d

In [13]:
input_directory = str(patient_path / "dicom")
output_directory = str(patient_path / "nifti")

print(input_directory)
print(output_directory)

# os.makedirs(output_directory, exist_ok=True)

# dicom2nifti.convert_directory(input_directory, output_directory)

../../dataset/v0/ONJ/EW-0003/MDCT/20191212/MDCT_axial/dicom
../../dataset/v0/ONJ/EW-0003/MDCT/20191212/MDCT_axial/nifti


In [ ]:
dicom2nifti.dicom_series_to_nifti(input_directory, output_directory)

In [11]:
print(input_directory)
print(output_directory)

../../dataset/v0/ONJ/EW-0003/MDCT/20191212/MDCT_axial/dicom
../../dataset/v0/ONJ/EW-0003/MDCT/20191212/MDCT_axial/nifti


In [ ]:
from pathlib import Path
import os
ONJ_PATH = DATAPATH / "ONJ"
NONONJ_PATH = DATAPATH / "NonONJ"
error_folders = []
CT_modals = ["MDCT", "CBCT"]

# Convert ONJ dicom file to nifti
for patient in os.listdir(ONJ_PATH):
    patient_path = ONJ_PATH / patient
    for modal in os.listdir(str(patient_path)):
        if modal in CT_modals:
            modal_path = patient_path / modal            
            modal_dir_path = list(modal_path.glob("*/*"))
            for folder in modal_dir_path:
                try:
                    os.makedirs(folder / "nifti", exist_ok=True)
                    print("Converting", folder / "dicom")
                    dicom2nifti.convert_directory(str(folder / "dicom"), str(folder / "nifti"))
                    print("Converted", folder / "nifti")
                except:
                    print("Error in converting", folder / "dicom")
                    error_folders.append(folder / "dicom")
                    continue


for patient in os.listdir(NONONJ_PATH):
    patient_path = NONONJ_PATH / patient
    for modal in os.listdir(str(patient_path)):
        if modal in CT_modals:
            modal_path = patient_path / modal            
            modal_dir_path = list(modal_path.glob("*/*"))
            for folder in modal_dir_path:
                try:
                    os.makedirs(folder / "nifti", exist_ok=True)
                    print("Converting", folder)
                    dicom2nifti.convert_directory(str(folder / "dicom"), str(folder / "nifti"))
                    print("Converted", folder)
                except:
                    print("Error in converting", folder)
                    continue


In [22]:
import pydicom
from PIL import Image
import numpy as np

dicom_folder = Path("/mnt/4TB1/onj/dataset/v0/ONJ/EW-0001/CBCT/20190724/CBCT_axial/dicom")
dicom_files = list(dicom_folder.glob("*"))
print(dicom_files)
# Load the compressed DICOM file

for dicom_file_name in dicom_files:
    dicom_file = pydicom.dcmread(dicom_file_name)

    # Extract the pixel data
    pixel_array = dicom_file.pixel_array

    # Convert to PIL Image (for demonstration, this assumes the pixel data is in a format that Pillow can handle directly)
    image = Image.fromarray(pixel_array)

    # If necessary, here you could adjust the image, change compression, etc.
    # For decompression, you might not need to do anything specific here,
    # but for changing compression or format, you would use Pillow's methods.

    # Convert back to numpy array if you've made changes with Pillow
    modified_pixel_array = np.array(image)

    # Update the DICOM file with the new uncompressed pixel data
    dicom_file.PixelData = modified_pixel_array.tobytes()

    # Update additional DICOM tags related to the image format if necessary
    # For example, changing the Transfer Syntax UID to indicate uncompressed data
    dicom_file.file_meta.TransferSyntaxUID = pydicom.uid.ImplicitVRLittleEndian

    # Save the modified DICOM file
    dicom_file.save_as(
        str(dicom_folder.parent / "dicom2" / dicom_file_name.name)
    )

[PosixPath('/mnt/4TB1/onj/dataset/v0/ONJ/EW-0001/CBCT/20190724/CBCT_axial/dicom/CB4.dcm'), PosixPath('/mnt/4TB1/onj/dataset/v0/ONJ/EW-0001/CBCT/20190724/CBCT_axial/dicom/CB11.dcm'), PosixPath('/mnt/4TB1/onj/dataset/v0/ONJ/EW-0001/CBCT/20190724/CBCT_axial/dicom/CB27.dcm'), PosixPath('/mnt/4TB1/onj/dataset/v0/ONJ/EW-0001/CBCT/20190724/CBCT_axial/dicom/CB35.dcm'), PosixPath('/mnt/4TB1/onj/dataset/v0/ONJ/EW-0001/CBCT/20190724/CBCT_axial/dicom/CB18.dcm'), PosixPath('/mnt/4TB1/onj/dataset/v0/ONJ/EW-0001/CBCT/20190724/CBCT_axial/dicom/CB17.dcm'), PosixPath('/mnt/4TB1/onj/dataset/v0/ONJ/EW-0001/CBCT/20190724/CBCT_axial/dicom/CB15.dcm'), PosixPath('/mnt/4TB1/onj/dataset/v0/ONJ/EW-0001/CBCT/20190724/CBCT_axial/dicom/CB36.dcm'), PosixPath('/mnt/4TB1/onj/dataset/v0/ONJ/EW-0001/CBCT/20190724/CBCT_axial/dicom/CB7.dcm'), PosixPath('/mnt/4TB1/onj/dataset/v0/ONJ/EW-0001/CBCT/20190724/CBCT_axial/dicom/CB30.dcm'), PosixPath('/mnt/4TB1/onj/dataset/v0/ONJ/EW-0001/CBCT/20190724/CBCT_axial/dicom/CB22.dcm'), 

In [23]:
patient_path = DATAPATH / "ONJ" / "EW-0001" / "CBCT" / "20190724" / "CBCT_axial"
print(patient_path)

input_directory = str(patient_path / "dicom2")
output_directory = str(patient_path / "nifti")

print(input_directory)
print(output_directory)

# os.makedirs(output_directory, exist_ok=True)

# dicom2nifti.convert_directory(input_directory, output_directory)

../../dataset/v0/ONJ/EW-0001/CBCT/20190724/CBCT_axial
../../dataset/v0/ONJ/EW-0001/CBCT/20190724/CBCT_axial/dicom2
../../dataset/v0/ONJ/EW-0001/CBCT/20190724/CBCT_axial/nifti


In [24]:
dicom2nifti.dicom_series_to_nifti(input_directory, output_directory)

/home/kim/anaconda3/envs/onj/lib/python3.10/site-packages/pydicom/filereader.py:353: UserWarning: Expected implicit VR, but found explicit VR - using explicit VR for reading
  warnings.warn(msg + f" - using {found_vr} VR for reading", UserWarning)


TypeError: float() argument must be a string or a real number, not 'NoneType'